Esta é uma script em jupyter para fazer pequenas funções e depois passar para a script em python.

In [ ]:
#Dar import a packages
from Bio import SeqIO, SeqFeature
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast import NCBIXML, NCBIWWW 

Função para fazer um NCBI blast apartir de um fasta de ume gene

In [ ]:
def gene_blast():
    pass